In [1]:
import pandas as pd

test = pd.read_parquet("../data/test.parquet")
item_feat = pd.read_parquet("../data/feat/item_feat.parquet")
recall = pd.read_parquet("../output/recalled.parquet")
recall

,session_type,labels
0,12899779,"[59625, 894169, 737445, 1246235, 499621, 46928..."
1,12899780,"[1142000, 736515, 973453, 582732, 1502122, 889..."
2,12899781,"[918667, 199008, 194067, 57315, 141736, 759436..."
3,12899782,"[1007613, 595994, 1033148, 834354, 479970, 169..."
4,12899783,"[1817895, 607638, 1754419, 1216820, 1729553, 3..."
...,...,...
1671798,14571577,"[1141710, 842555, 1276792, 1478287, 934235, 17..."
1671799,14571578,"[519105, 977826, 1811714, 815460, 822641, 1671..."
1671800,14571579,"[739876, 1209992, 1750859, 1550479, 785544, 11..."
1671801,14571580,"[202353, 682237, 336919, 848099, 1482884, 9256..."


In [2]:
# 每个 session 的点击/加购/购买数量
counts = test.groupby(['session', 'type']).size().unstack(fill_value=0)

# 重命名列（0=click, 1=cart, 2=buy）
counts = counts.rename(columns={0: "click_num", 1: "cart_num", 2: "buy_num"}).reset_index()
counts

type,session,click_num,cart_num,buy_num
0,12899779,1,0,0
1,12899780,5,0,0
2,12899781,10,1,0
3,12899782,46,16,8
4,12899783,11,0,0
...,...,...,...,...
1671798,14571577,1,0,0
1671799,14571578,1,0,0
1671800,14571579,1,0,0
1671801,14571580,1,0,0


In [3]:
test = test.merge(counts, on="session", how="left")
test

,session,aid,ts,type,click_num,cart_num,buy_num
0,12899779,59625,1661724000,0,1,0,0
1,12899780,1142000,1661724000,0,5,0,0
2,12899780,582732,1661724058,0,5,0,0
3,12899780,973453,1661724109,0,5,0,0
4,12899780,736515,1661724136,0,5,0,0
...,...,...,...,...,...,...,...
6928118,14571577,1141710,1662328774,0,1,0,0
6928119,14571578,519105,1662328775,0,1,0,0
6928120,14571579,739876,1662328775,0,1,0,0
6928121,14571580,202353,1662328781,0,1,0,0


In [4]:
# session 长度
test['session_length'] = test.groupby('session')['session'].transform('size')

# 比例特征
test['click_ratio'] = test['click_num'] / test['session_length']
test['cart_ratio']  = test['cart_num']  / test['session_length']
test['buy_ratio']   = test['buy_num']   / test['session_length']

# 时间跨度
test['time_diff'] = test.groupby('session')['ts'].transform(lambda x: x.max() - x.min())

# 去重的 item 数
test['item_count'] = test.groupby('session')['aid'].transform('nunique')

In [5]:
test

,session,aid,ts,type,click_num,cart_num,buy_num,session_length,click_ratio,cart_ratio,buy_ratio,time_diff,item_count
0,12899779,59625,1661724000,0,1,0,0,1,1.0,0.0,0.0,0,1
1,12899780,1142000,1661724000,0,5,0,0,5,1.0,0.0,0.0,155,4
2,12899780,582732,1661724058,0,5,0,0,5,1.0,0.0,0.0,155,4
3,12899780,973453,1661724109,0,5,0,0,5,1.0,0.0,0.0,155,4
4,12899780,736515,1661724136,0,5,0,0,5,1.0,0.0,0.0,155,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6928118,14571577,1141710,1662328774,0,1,0,0,1,1.0,0.0,0.0,0,1
6928119,14571578,519105,1662328775,0,1,0,0,1,1.0,0.0,0.0,0,1
6928120,14571579,739876,1662328775,0,1,0,0,1,1.0,0.0,0.0,0,1
6928121,14571580,202353,1662328781,0,1,0,0,1,1.0,0.0,0.0,0,1


In [6]:
sesssion_feat = test.drop_duplicates("session")[['session','click_num','cart_num','buy_num','item_count','session_length','click_ratio','cart_ratio','buy_ratio','time_diff']]
sesssion_feat

,session,click_num,cart_num,buy_num,item_count,session_length,click_ratio,cart_ratio,buy_ratio,time_diff
0,12899779,1,0,0,1,1,1.000000,0.000000,0.000000,0
1,12899780,5,0,0,4,5,1.000000,0.000000,0.000000,155
6,12899781,10,1,0,5,11,0.909091,0.090909,0.000000,336160
17,12899782,46,16,8,38,70,0.657143,0.228571,0.114286,79953
87,12899783,11,0,0,9,11,1.000000,0.000000,0.000000,317140
...,...,...,...,...,...,...,...,...,...,...
6928118,14571577,1,0,0,1,1,1.000000,0.000000,0.000000,0
6928119,14571578,1,0,0,1,1,1.000000,0.000000,0.000000,0
6928120,14571579,1,0,0,1,1,1.000000,0.000000,0.000000,0
6928121,14571580,1,0,0,1,1,1.000000,0.000000,0.000000,0


In [7]:
recalled = recall.explode("labels")
recalled

,session_type,labels
0,12899779,59625
0,12899779,894169
0,12899779,737445
0,12899779,1246235
0,12899779,499621
...,...,...
1671802,14571581,147545
1671802,14571581,1409106
1671802,14571581,204740
1671802,14571581,149039


In [8]:
recalled = recalled.rename(columns={"session_type":"session","labels":"aid"})

In [9]:
recalled_feature = recalled.merge(sesssion_feat,on="session",how="left")
recalled_feature = recalled_feature.merge(item_feat,on="aid",how="left").fillna(0)
recalled_feature

/tmp/ipykernel_237062/196290352.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalled_feature = recalled_feature.merge(item_feat,on="aid",how="left").fillna(0)


,session,aid,click_num,cart_num,buy_num,item_count,session_length,click_ratio,cart_ratio,buy_ratio,time_diff,item_click_count,item_carts_count,item_orders_count
0,12899779,59625,1,0,0,1,1,1.0,0.0,0.0,0,12,0.0,0.0
1,12899779,894169,1,0,0,1,1,1.0,0.0,0.0,0,60,1.0,0.0
2,12899779,737445,1,0,0,1,1,1.0,0.0,0.0,0,909,15.0,0.0
3,12899779,1246235,1,0,0,1,1,1.0,0.0,0.0,0,2172,94.0,8.0
4,12899779,499621,1,0,0,1,1,1.0,0.0,0.0,0,29,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460279690,14571581,147545,1,0,0,1,1,1.0,0.0,0.0,0,153,12.0,0.0
460279691,14571581,1409106,1,0,0,1,1,1.0,0.0,0.0,0,3525,259.0,43.0
460279692,14571581,204740,1,0,0,1,1,1.0,0.0,0.0,0,6585,485.0,82.0
460279693,14571581,149039,1,0,0,1,1,1.0,0.0,0.0,0,8575,226.0,6.0


In [10]:
recalled_feature['rank_in_co_vis_recall'] = recalled_feature.groupby("session").cumcount() + 1
recalled_feature

,session,aid,click_num,cart_num,buy_num,item_count,session_length,click_ratio,cart_ratio,buy_ratio,time_diff,item_click_count,item_carts_count,item_orders_count,rank_in_co_vis_recall
0,12899779,59625,1,0,0,1,1,1.0,0.0,0.0,0,12,0.0,0.0,1
1,12899779,894169,1,0,0,1,1,1.0,0.0,0.0,0,60,1.0,0.0,2
2,12899779,737445,1,0,0,1,1,1.0,0.0,0.0,0,909,15.0,0.0,3
3,12899779,1246235,1,0,0,1,1,1.0,0.0,0.0,0,2172,94.0,8.0,4
4,12899779,499621,1,0,0,1,1,1.0,0.0,0.0,0,29,1.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460279690,14571581,147545,1,0,0,1,1,1.0,0.0,0.0,0,153,12.0,0.0,296
460279691,14571581,1409106,1,0,0,1,1,1.0,0.0,0.0,0,3525,259.0,43.0,297
460279692,14571581,204740,1,0,0,1,1,1.0,0.0,0.0,0,6585,485.0,82.0,298
460279693,14571581,149039,1,0,0,1,1,1.0,0.0,0.0,0,8575,226.0,6.0,299


In [11]:
import joblib

model = {}
for t in ['clicks','carts','orders'] :
    model[t] = joblib.load(f"../output/ranker_{t}.pkl")

In [12]:
# 预测打分
preds = []

for t in ['clicks', 'carts', 'orders']:
    print(f"Scoring for {t} ...")
    # 当前 session_type 的数据
    df_t = recalled_feature.copy()
    df_t['session_type'] = df_t['session'].astype(str) + f"_{t}"

    # 提取特征（去掉 id 字段）
    drop_cols = ['session', 'aid', 'session_type']
    X = df_t.drop(columns=drop_cols, errors="ignore")

    # 模型预测
    y_pred = model[t].predict(X)  # 直接得到排序分数

    df_t['pred'] = y_pred

    preds.append(df_t)

Scoring for clicks ...
Scoring for carts ...
Scoring for orders ...


In [13]:
# 合并三个结果
all_preds = pd.concat(preds, ignore_index=True)
all_preds

,session,aid,click_num,cart_num,buy_num,item_count,session_length,click_ratio,cart_ratio,buy_ratio,time_diff,item_click_count,item_carts_count,item_orders_count,rank_in_co_vis_recall,session_type,pred
0,12899779,59625,1,0,0,1,1,1.0,0.0,0.0,0,12,0.0,0.0,1,12899779_clicks,4.257342
1,12899779,894169,1,0,0,1,1,1.0,0.0,0.0,0,60,1.0,0.0,2,12899779_clicks,4.059726
2,12899779,737445,1,0,0,1,1,1.0,0.0,0.0,0,909,15.0,0.0,3,12899779_clicks,0.372548
3,12899779,1246235,1,0,0,1,1,1.0,0.0,0.0,0,2172,94.0,8.0,4,12899779_clicks,0.191566
4,12899779,499621,1,0,0,1,1,1.0,0.0,0.0,0,29,1.0,0.0,5,12899779_clicks,-1.292145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380839080,14571581,147545,1,0,0,1,1,1.0,0.0,0.0,0,153,12.0,0.0,296,14571581_orders,-7.633316
1380839081,14571581,1409106,1,0,0,1,1,1.0,0.0,0.0,0,3525,259.0,43.0,297,14571581_orders,-5.519159
1380839082,14571581,204740,1,0,0,1,1,1.0,0.0,0.0,0,6585,485.0,82.0,298,14571581_orders,-5.339006
1380839083,14571581,149039,1,0,0,1,1,1.0,0.0,0.0,0,8575,226.0,6.0,299,14571581_orders,-6.403477


In [14]:
all_preds_res = all_preds[['session_type','aid','pred']]
all_preds_res

,session_type,aid,pred
0,12899779_clicks,59625,4.257342
1,12899779_clicks,894169,4.059726
2,12899779_clicks,737445,0.372548
3,12899779_clicks,1246235,0.191566
4,12899779_clicks,499621,-1.292145
...,...,...,...
1380839080,14571581_orders,147545,-7.633316
1380839081,14571581_orders,1409106,-5.519159
1380839082,14571581_orders,204740,-5.339006
1380839083,14571581_orders,149039,-6.403477


In [16]:
gps = all_preds_res.groupby("session_type")

In [21]:
all_preds_res.to_parquet("../output/all_preds.parquet")

In [26]:
from tqdm import tqdm
import pandas as pd

# 全局 tqdm 计数器
pbar = tqdm(total=3*test['session'].nunique(), desc="Processing groups")

def process_group(x):
    # 每调用一次就更新进度条
    pbar.update(1)
    return x.sort_values("pred", ascending=False).head(20)

res = all_preds_res.groupby("session_type").apply(process_group).reset_index(drop=True)
pbar.close()


Processing groups:   0%|          | 0/1671803 [00:00<?, ?it/s]/tmp/ipykernel_237062/479849137.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  res = all_preds_res.groupby("session_type").apply(process_group).reset_index(drop=True)
Processing groups: 5015409it [28:47, 4863.96it/s]                              /tmp/ipykernel_237062/479849137.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  res = all_preds_res.groupby("session_type").apply(process_group).reset_index(drop=True)
Processing groups: 5015409

In [27]:
# 只对 aid 列变成 list
submission = res.groupby("session_type")["aid"].agg(list).reset_index()
submission = submission.rename(columns={"aid": "labels"})
submission

/tmp/ipykernel_237062/3069073259.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  submission = res.groupby("session_type")["aid"].agg(list).reset_index()


,session_type,labels
0,12899779_carts,"[894169, 737445, 1246235, 469285, 499621, 9415..."
1,12899779_clicks,"[59625, 894169, 737445, 1246235, 469285, 49962..."
2,12899779_orders,"[1246235, 731692, 941596, 1790770, 1474359, 17..."
3,12899780_carts,"[973453, 582732, 736515, 1142000, 889686, 1515..."
4,12899780_clicks,"[736515, 1142000, 582732, 973453, 889686, 1502..."
...,...,...
5015404,14571580_clicks,"[202353, 682237, 848099, 336919, 1482884, 9256..."
5015405,14571580_orders,"[202353, 682237, 336919, 848099, 925638, 13145..."
5015406,14571581_carts,"[622489, 1100210, 1684953, 462056, 1571699, 11..."
5015407,14571581_clicks,"[1100210, 1684953, 622489, 462056, 1571699, 11..."


In [30]:
submission['labels'] = submission['labels'].apply(lambda x : ' '.join(map(str,x)))

In [31]:
submission.to_csv("../output/submission.csv", index=False)

In [32]:
submission

,session_type,labels
0,12899779_carts,894169 737445 1246235 469285 499621 941596 731...
1,12899779_clicks,59625 894169 737445 1246235 469285 499621 9415...
2,12899779_orders,1246235 731692 941596 1790770 1474359 1770962 ...
3,12899780_carts,973453 582732 736515 1142000 889686 1515511 48...
4,12899780_clicks,736515 1142000 582732 973453 889686 1502122 15...
...,...,...
5015404,14571580_clicks,202353 682237 848099 336919 1482884 925638 433...
5015405,14571580_orders,202353 682237 336919 848099 925638 1314572 433...
5015406,14571581_carts,622489 1100210 1684953 462056 1571699 1158237 ...
5015407,14571581_clicks,1100210 1684953 622489 462056 1571699 1158237 ...
